# Cleaning Players

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import seaborn as sns

In [170]:
players = pd.read_csv("players.csv")
players.columns = players.columns.str.replace(' ', '')

In [171]:
players

,nflId,height,weight,birthDate,collegeName,Position,displayName
0,42901,6-1,208,1992-07-25,James Madison,SS,Dean Marlowe
1,43501,6-0,220,1994-08-11,Central Michigan,FS,Kavon Frazier
2,43534,6-0,205,1993-06-02,Illinois,SS,Clayton Fejedelem
3,43535,6-1,235,1992-12-22,Temple,MLB,Tyler Matakevich
4,44174,6-1,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen
...,...,...,...,...,...,...,...
2727,33216,6-3,320,06/16/1986,Central Florida,G,Josh Sitton
2728,33184,6-3,265,05/02/1985,Winston-Salem State,DE,William Hayes
2729,45651,5-9,198,06/16/1993,Iowa,WR,Riley McCarron
2730,41284,73,230,1992-10-20,LSU,RB,Jeremy Hill


Converting all heights to inches

In [173]:
check = players['height'].str.split('-', expand=True)

check.columns = ['feet', 'inches']

check.loc[(check['inches'].notnull()), 'feet'] = check[check['inches'].notnull()]['feet'].astype(np.int16) * 12 + check[check['inches'].notnull()]['inches'].astype(np.int16)



In [174]:
players['height'] = check['feet']
players['height'] = players['height'].astype(np.float32)
players


,nflId,height,weight,birthDate,collegeName,Position,displayName
0,42901,73.0,208,1992-07-25,James Madison,SS,Dean Marlowe
1,43501,72.0,220,1994-08-11,Central Michigan,FS,Kavon Frazier
2,43534,72.0,205,1993-06-02,Illinois,SS,Clayton Fejedelem
3,43535,73.0,235,1992-12-22,Temple,MLB,Tyler Matakevich
4,44174,73.0,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen
...,...,...,...,...,...,...,...
2727,33216,75.0,320,06/16/1986,Central Florida,G,Josh Sitton
2728,33184,75.0,265,05/02/1985,Winston-Salem State,DE,William Hayes
2729,45651,69.0,198,06/16/1993,Iowa,WR,Riley McCarron
2730,41284,73.0,230,1992-10-20,LSU,RB,Jeremy Hill


Making all dates the same format

In [175]:
for idx, row in players.iterrows():
  if "/" in row['birthDate']: 
        split = row["birthDate"].split("/")
        players.loc[idx,"birthDate"] = split[2].replace(" ","")+"-"+split[0]+"-"+split[1]

In [176]:
players

,nflId,height,weight,birthDate,collegeName,Position,displayName
0,42901,73.0,208,1992-07-25,James Madison,SS,Dean Marlowe
1,43501,72.0,220,1994-08-11,Central Michigan,FS,Kavon Frazier
2,43534,72.0,205,1993-06-02,Illinois,SS,Clayton Fejedelem
3,43535,73.0,235,1992-12-22,Temple,MLB,Tyler Matakevich
4,44174,73.0,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen
...,...,...,...,...,...,...,...
2727,33216,75.0,320,1986-06-16,Central Florida,G,Josh Sitton
2728,33184,75.0,265,1985-05-02,Winston-Salem State,DE,William Hayes
2729,45651,69.0,198,1993-06-16,Iowa,WR,Riley McCarron
2730,41284,73.0,230,1992-10-20,LSU,RB,Jeremy Hill


In [178]:
players.to_csv("cleaned_players.csv",index=False)

In [180]:
cleaned_players = pd.read_csv("cleaned_players.csv")
cleaned_players

,nflId,height,weight,birthDate,collegeName,Position,displayName
0,42901,73.0,208,1992-07-25,James Madison,SS,Dean Marlowe
1,43501,72.0,220,1994-08-11,Central Michigan,FS,Kavon Frazier
2,43534,72.0,205,1993-06-02,Illinois,SS,Clayton Fejedelem
3,43535,73.0,235,1992-12-22,Temple,MLB,Tyler Matakevich
4,44174,73.0,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen
...,...,...,...,...,...,...,...
2727,33216,75.0,320,1986-06-16,Central Florida,G,Josh Sitton
2728,33184,75.0,265,1985-05-02,Winston-Salem State,DE,William Hayes
2729,45651,69.0,198,1993-06-16,Iowa,WR,Riley McCarron
2730,41284,73.0,230,1992-10-20,LSU,RB,Jeremy Hill


# Plays

In [196]:
plays = pd.read_csv("plays.csv")
plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
0,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,NaN,NaN,NaN,0,0,NaN,66.0,NaN,40,45
1,2018090600,366,"(9:20) C.Johnston punts 56 yards to ATL 36, Ce...",1,4,4,PHI,Punt,Return,45603.0,...,UNSd,PHI 18,-15.0,0,0,NaN,56.0,5.0,36,18
2,2018090600,658,"(5:03) M.Bryant 21 yard field goal is GOOD, Ce...",1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,...,NaN,NaN,NaN,0,0,NaN,21.0,NaN,0,13
3,2018090600,677,M.Bosher kicks 64 yards from ATL 35 to PHI 1. ...,1,0,0,ATL,Kickoff,Return,37267.0,...,NaN,NaN,NaN,0,3,NaN,64.0,30.0,34,75
4,2018090600,872,"(:33) C.Johnston punts 65 yards to end zone, C...",1,4,18,PHI,Punt,Touchback,45603.0,...,NaN,NaN,NaN,0,3,NaN,65.0,NaN,45,45


There are four special plays detailed. They should be given their own csvs.

In [187]:
plays['specialTeamsPlayType'].unique()

array(['Kickoff', 'Punt', 'Field Goal', 'Extra Point'], dtype=object)

In [190]:
plays[plays['specialTeamsPlayType'] == "Kickoff"]["specialTeamsResult"].unique()

array(['Touchback', 'Return', 'Muffed', 'Kickoff Team Recovery',
       'Out of Bounds', 'Fair Catch', 'Downed'], dtype=object)

- Touchback - Kickoff resulted in ball becoming dead in defending team's endzone, so defending team gain possesion at 25 or 20 yard line. Either has to land there and stop, or a player catches and kneels to end play.
- Return - Kickoff resulted in ball being received by defending team and them running the ball up the field. (Is caught or becomes dead not in end zone?)
- Muffed - Receiving team don't gain possession of the ball properly, and can only start at where the ball was downed?
- Kickoff Team Recovery - kickoff team gain possesion of the ball after it crosses the receiving team's restraining line (35 yards) or a member of the receiving team possess the ball first.
- Out of Bounds - out of bounds
- Fair Catch - Receiver signals that they want a fair catch, meaning they can catch the ball without interference. Then the ball becomes dead at that spot and the receiving team cannot advance it.
- Downed - Ball brought to the ground??

In [191]:
plays[plays['specialTeamsPlayType'] == "Punt"]["specialTeamsResult"].unique()

array(['Return', 'Touchback', 'Fair Catch', 'Downed', 'Muffed',
       'Out of Bounds', 'Non-Special Teams Result', 'Blocked Punt'],
      dtype=object)

- Non-Special Teams Result - Punt is passed instead.

In [192]:
plays[plays['specialTeamsPlayType'] == "Field Goal"]["specialTeamsResult"].unique()

array(['Kick Attempt Good', 'Kick Attempt No Good',
       'Blocked Kick Attempt', 'Non-Special Teams Result',
       'Out of Bounds', 'Downed'], dtype=object)

- Kick Attempt Good - goal scored
- Kick Attempt No Good - goal missed
- Blocked Kick Attempt - kick blocked by an opponent
- Non-Special Teams Result - kick set up but passed instead?

In [193]:
plays[plays['specialTeamsPlayType'] == "Extra Point"]["specialTeamsResult"].unique()

array(['Kick Attempt Good', 'Kick Attempt No Good',
       'Non-Special Teams Result', 'Blocked Kick Attempt'], dtype=object)

- Non-Special Teams Result - Can choose to attempt another touchdown after first touchdown instead of conversion kick, so no one attempts the kick, kickerId is null. Mostly fails however.

## Kickoff

In [217]:
kickoff = plays[plays['specialTeamsPlayType'] == "Kickoff"]
kickoff.columns

Index(['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'specialTeamsPlayType', 'specialTeamsResult',
       'kickerId', 'returnerId', 'kickBlockerId', 'yardlineSide',
       'yardlineNumber', 'gameClock', 'penaltyCodes', 'penaltyJerseyNumbers',
       'penaltyYards', 'preSnapHomeScore', 'preSnapVisitorScore', 'passResult',
       'kickLength', 'kickReturnYardage', 'playResult',
       'absoluteYardlineNumber'],
      dtype='object')

The percentage of NA values in each column:

In [210]:
for column in kickoff.columns:
  print(column,(kickoff[column].isnull().sum()/len(kickoff[column])*100))

gameId 0.0
playId 0.0
playDescription 0.0
quarter 0.0
down 0.0
yardsToGo 0.0
possessionTeam 0.0
specialTeamsPlayType 0.0
specialTeamsResult 0.0
kickerId 0.0
returnerId 63.55986229759021
kickBlockerId 100.0
yardlineSide 0.7267627183475711
yardlineNumber 0.0
gameClock 0.0
penaltyCodes 95.4609205661099
penaltyJerseyNumbers 95.4609205661099
penaltyYards 95.4609205661099
preSnapHomeScore 0.0
preSnapVisitorScore 0.0
passResult 100.0
kickLength 0.0
kickReturnYardage 62.14458753028178
playResult 0.0
absoluteYardlineNumber 0.0


- Penalties have high percentages because they are rare, but still valid data
- Kickoffs have no kick blocker so kickBlockerId is irrelevant here
- passResult: Scrimmage outcome of the play if specialTeamsPlayResult is "Non-Special Teams Result", so irrelevant here
- looks like yardlineNumber should all be 35 because that's where a kickoff occurs, but some maybe different because of pentalies?

In [218]:
kickoff = kickoff.drop(columns=["kickBlockerId","passResult","specialTeamsPlayType"])

In [220]:
kickoff.to_csv("kickoff.csv",index=False)

specialTeamsPlayType is removed because the csv only has data about one special type, so would be a column with all the same values

## Punt

In [222]:
punt = plays[plays['specialTeamsPlayType'] == "Punt"]
punt

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
1,2018090600,366,"(9:20) C.Johnston punts 56 yards to ATL 36, Ce...",1,4,4,PHI,Punt,Return,45603.0,...,UNSd,PHI 18,-15.0,0,0,NaN,56.0,5.0,36,18
4,2018090600,872,"(:33) C.Johnston punts 65 yards to end zone, C...",1,4,18,PHI,Punt,Touchback,45603.0,...,NaN,NaN,NaN,0,3,NaN,65.0,NaN,45,45
5,2018090600,973,"(14:18) M.Bosher punts 49 yards to PHI 40, Cen...",2,4,19,ATL,Punt,Fair Catch,37267.0,...,NaN,NaN,NaN,0,3,NaN,49.0,NaN,49,21
10,2018090600,1850,"(1:30) C.Johnston punts 58 yards to ATL 11, Ce...",2,4,8,PHI,Punt,Downed,45603.0,...,NaN,NaN,NaN,3,6,NaN,58.0,NaN,58,79
11,2018090600,1989,"(:44) M.Bosher punts 57 yards to PHI 17, Cente...",2,4,8,ATL,Punt,Return,37267.0,...,UNR,PHI 50,12.0,3,6,NaN,57.0,8.0,61,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19957,2021010315,1359,"(8:17) M.Wishnowsky punts 45 yards to SEA 35, ...",2,4,15,SF,Punt,Return,47893.0,...,IBW,SEA 18,10.0,0,6,NaN,45.0,7.0,48,90
19958,2021010315,1479,"(6:47) M.Dickson punts 61 yards to SF 16, Cent...",2,4,19,SEA,Punt,Return,46218.0,...,NaN,NaN,NaN,0,6,NaN,61.0,5.0,56,33
19964,2021010315,2235,"(8:39) M.Dickson punts 43 yards to SF 30, Cent...",3,4,8,SEA,Punt,Return,46218.0,...,NaN,NaN,NaN,6,6,NaN,43.0,21.0,22,37
19967,2021010315,2497,"(5:17) M.Dickson punts 52 yards to SF 22, Cent...",3,4,9,SEA,Punt,Return,46218.0,...,NaN,NaN,NaN,9,6,NaN,52.0,5.0,47,36


In [225]:
for column in punt.columns:
  print(column,(punt[column].isnull().sum()/len(punt[column])*100))

gameId 0.0
playId 0.0
playDescription 0.0
quarter 0.0
down 0.0
yardsToGo 0.0
possessionTeam 0.0
specialTeamsPlayType 0.0
specialTeamsResult 0.0
kickerId 1.2351861125020864
returnerId 31.931230178601233
kickBlockerId 99.34902353530295
yardlineSide 1.8193957603071274
yardlineNumber 0.0
gameClock 0.0
penaltyCodes 89.30061759305626
penaltyJerseyNumbers 89.30061759305626
penaltyYards 89.30061759305626
preSnapHomeScore 0.0
preSnapVisitorScore 0.0
passResult 99.48255716908696
kickLength 1.8861625771991322
kickReturnYardage 60.691036554832245
playResult 0.0
absoluteYardlineNumber 0.0


- Some kickerIds are null because the punt is not kicked (??), it is passed instead. Indicated by having the specialTeamsResult set to Non-Special Teams Result, and then the passResult shows the result of the pass.
- kickBlockerId is mostly null because it is rare to block a punt. When not null, specialTeamsResult has Blocked Punt


In [226]:
punt = punt.drop(columns=["specialTeamsPlayType"])

In [227]:
punt.to_csv("punt.csv",index=False)

## Field Goal

In [240]:
fieldGoal = plays[plays['specialTeamsPlayType'] == "Field Goal"]
fieldGoal

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
2,2018090600,658,"(5:03) M.Bryant 21 yard field goal is GOOD, Ce...",1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,...,NaN,NaN,NaN,0,0,NaN,21.0,NaN,0,13
6,2018090600,1368,"(6:12) J.Elliott 26 yard field goal is GOOD, C...",2,4,8,PHI,Field Goal,Kick Attempt Good,44966.0,...,NaN,NaN,NaN,0,3,NaN,26.0,NaN,0,18
8,2018090600,1587,"(2:13) M.Bryant 52 yard field goal is GOOD, Ce...",2,4,17,ATL,Field Goal,Kick Attempt Good,27091.0,...,NaN,NaN,NaN,3,3,NaN,52.0,NaN,0,76
33,2018090900,1226,(13:18) (Field Goal formation) J.Tucker 41 yar...,2,4,13,BAL,Field Goal,Kick Attempt Good,39470.0,...,NaN,NaN,NaN,14,0,NaN,41.0,NaN,0,33
36,2018090900,1613,(8:48) (Field Goal formation) S.Hauschka 52 ya...,2,4,9,BUF,Field Goal,Kick Attempt No Good,33469.0,...,NaN,NaN,NaN,17,0,NaN,52.0,NaN,-8,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19952,2021010315,811,"(2:13) J.Myers 36 yard field goal is GOOD, Cen...",1,4,10,SEA,Field Goal,Kick Attempt Good,41175.0,...,NaN,NaN,NaN,0,0,NaN,36.0,NaN,0,27
19955,2021010315,1243,"(9:31) J.Myers 30 yard field goal is GOOD, Cen...",2,4,3,SEA,Field Goal,Kick Attempt Good,41175.0,...,NaN,NaN,NaN,0,3,NaN,30.0,NaN,0,98
19959,2021010315,1784,"(:29) T.Vizcaino 36 yard field goal is GOOD, C...",2,4,6,SF,Field Goal,Kick Attempt Good,47590.0,...,NaN,NaN,NaN,0,6,NaN,36.0,NaN,0,28
19962,2021010315,2121,"(9:36) T.Vizcaino 47 yard field goal is GOOD, ...",3,4,8,SF,Field Goal,Kick Attempt Good,47590.0,...,NaN,NaN,NaN,3,6,NaN,47.0,NaN,0,39


In [232]:
for column in fieldGoal.columns:
  print(column,(fieldGoal[column].isnull().sum()/len(fieldGoal[column])*100))

gameId 0.0
playId 0.0
playDescription 0.0
quarter 0.0
down 0.0
yardsToGo 0.0
possessionTeam 0.0
specialTeamsPlayType 0.0
specialTeamsResult 0.0
kickerId 0.5269100489273617
returnerId 99.92472713586751
kickBlockerId 98.6074520135491
yardlineSide 0.0
yardlineNumber 0.0
gameClock 0.0
penaltyCodes 98.4945427173504
penaltyJerseyNumbers 98.4945427173504
penaltyYards 98.4945427173504
preSnapHomeScore 0.0
preSnapVisitorScore 0.0
passResult 99.6236356793376
kickLength 1.919458035378246
kickReturnYardage 100.0
playResult 0.0
absoluteYardlineNumber 0.0


- kickReturnYardage is all null because the receiving cannot (??) advance the ball after a field goal ??
- playResult is mostly 0 because most attempts score goals, so kicking team essentially gains no yards because play is reset. Will be negative if goal is missed so receiving team get the ball at their 8 yard mark (??). For blocked kicks, it's anyone's ball after so kicking team may or may not gain yards afterwards.
- returnerId is mostly null because it's rare to return after a field goal??

In [241]:
fieldGoal = fieldGoal.drop(columns=["specialTeamsPlayType","kickReturnYardage"])

In [242]:
fieldGoal.to_csv("fieldGoal.csv",index=False)

# Extra Point

In [236]:
extraPoint = plays[plays['specialTeamsPlayType'] == "Extra Point"]
extraPoint

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
15,2018090600,2883,"J.Elliott extra point is GOOD, Center-R.Lovato...",3,0,0,PHI,Extra Point,Kick Attempt Good,44966.0,...,NaN,NaN,NaN,9,6,NaN,NaN,NaN,0,25
19,2018090600,3553,"M.Bryant extra point is No Good, Hit Right Upr...",4,0,0,ATL,Extra Point,Kick Attempt No Good,27091.0,...,NaN,NaN,NaN,10,12,NaN,NaN,NaN,0,25
25,2018090900,380,"J.Tucker extra point is GOOD, Center-M.Cox, Ho...",1,0,0,BAL,Extra Point,Kick Attempt Good,39470.0,...,NaN,NaN,NaN,6,0,NaN,NaN,NaN,0,95
30,2018090900,972,"J.Tucker extra point is GOOD, Center-M.Cox, Ho...",1,0,0,BAL,Extra Point,Kick Attempt Good,39470.0,...,NaN,NaN,NaN,13,0,NaN,NaN,NaN,0,95
44,2018090900,2757,"J.Tucker extra point is GOOD, Center-M.Cox, Ho...",3,0,0,BAL,Extra Point,Kick Attempt Good,39470.0,...,NaN,NaN,NaN,32,0,NaN,NaN,NaN,0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19968,2021010315,2813,"T.Vizcaino extra point is GOOD, Center-C.Holba...",4,0,0,SF,Extra Point,Kick Attempt Good,47590.0,...,NaN,NaN,NaN,15,6,NaN,NaN,NaN,0,95
19970,2021010315,3074,"J.Myers extra point is No Good, Wide Left, Cen...",4,0,0,SEA,Extra Point,Kick Attempt No Good,41175.0,...,NaN,NaN,NaN,16,12,NaN,NaN,NaN,0,25
19973,2021010315,3667,"J.Myers extra point is GOOD, Center-T.Ott, Hol...",4,0,0,SEA,Extra Point,Kick Attempt Good,41175.0,...,NaN,NaN,NaN,16,18,NaN,NaN,NaN,0,25
19975,2021010315,3870,"J.Myers extra point is GOOD, Center-T.Ott, Hol...",4,0,0,SEA,Extra Point,Kick Attempt Good,41175.0,...,NaN,NaN,NaN,16,25,NaN,NaN,NaN,0,25


In [237]:
for column in extraPoint.columns:
  print(column,(extraPoint[column].isnull().sum()/len(extraPoint[column])*100))

gameId 0.0
playId 0.0
playDescription 0.0
quarter 0.0
down 0.0
yardsToGo 0.0
possessionTeam 0.0
specialTeamsPlayType 0.0
specialTeamsResult 0.0
kickerId 0.37270642201834864
returnerId 100.0
kickBlockerId 99.31192660550458
yardlineSide 0.0
yardlineNumber 0.0
gameClock 0.0
penaltyCodes 98.13646788990825
penaltyJerseyNumbers 98.13646788990825
penaltyYards 98.13646788990825
preSnapHomeScore 0.0
preSnapVisitorScore 0.0
passResult 99.88532110091744
kickLength 100.0
kickReturnYardage 100.0
playResult 0.0
absoluteYardlineNumber 0.0


- returnerId all null because no one returns
- kickLength all null because kicks happen at same place
- kickReturnYardage all null because you can't advance after an extra point attempt

In [243]:
extraPoint = extraPoint.drop(columns=["specialTeamsPlayType","kickReturnYardage","returnerId","kickLength"])

In [244]:
extraPoint.to_csv("extraPoint.csv",index=False)